<a href="https://colab.research.google.com/github/YuvanBharathi13/DeepLearning/blob/main/4_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

AttributeError: module 'numpy' has no attribute 'dtypes'

# Knowing the data

## unzipping the file

In [2]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2025-06-13 14:44:31--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 142.250.99.207, 142.250.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   206MB/s    in 0.8s    

2025-06-13 14:44:32 (206 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



## walking through the directory

In [3]:
import os
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} folders and {len(filenames)} images in directory {dirpath}")

There are 2 folders and 0 images in directory 10_food_classes_10_percent
There are 10 folders and 0 images in directory 10_food_classes_10_percent/train
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/pizza
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_curry
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/grilled_salmon
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/hamburger
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/ice_cream
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/steak
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/fried_rice
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_wings
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/sushi
There are 0 folders and 75 images i

## class names

In [4]:
import pathlib
data_dir = pathlib.Path('10_food_classes_10_percent/train')
class_names = np.array(sorted(item.name for item in data_dir.glob('*')))
class_names

array(['chicken_curry', 'chicken_wings', 'fried_rice', 'grilled_salmon',
       'hamburger', 'ice_cream', 'pizza', 'ramen', 'steak', 'sushi'],
      dtype='<U14')

# Pre-processing data

In [5]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


# Setting up callbacks

In [7]:
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Creating models using tensorflow hub

- Feature extraction transfer learning is when you take the underlying patterns (also called weights) a pretrained model has learned and adjust its outputs to be more suited to your problem.
- For example, say the pretrained model you were using had 236 different layers (EfficientNetB0 has 236 layers), but the top layer outputs 1000 classes because it was pretrained on ImageNet. To adjust this to your own problem, you might remove the original activation layer and replace it with your own but with the right number of output classes. The important part here is that only the top few layers become trainable, the rest remain frozen.
- A feature extraction model only the top 2-3 layers change and in a fine-tuning model many or all of original model get changed.

In [31]:
import tensorflow_hub as hub

In [32]:
resnet_url = "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/2"

efficientnet_url = "https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1"

In [33]:
def create_model(model_url, num_classes=10):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=(224,224,3)) # define the input image shape
  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer
  ])

  return model

In [34]:
efficientnet_model = create_model(efficientnet_url, num_classes=train_data.num_classes)

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7e6dafc291d0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [35]:
print("TF version:", tf.__version__)
print("TF Hub version:", hub.__version__)

TF version: 2.18.0
TF Hub version: 0.16.1
